In [1]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [2]:
import numpy as np
import cv2
import copy
from calibration import generate_truth_locations, recover_homographies
from util import *


class Camera:
    def __init__(self) -> None:
        pass

    @staticmethod
    def static_calibrate(gray_images: np.ndarray, chessboard_size, chessboard_cell_size, save_image=True, save_path='./images/') -> np.ndarray:
        # H矩阵
        H_s = []
        for i in range(gray_images.shape[0]):
            # 寻找棋盘格角点的模式 自适应阈值+标准化图片
            chessboard_flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE

            ret, corners = cv2.findChessboardCorners(
                gray_images[i], chessboard_size, chessboard_flags)

            # 若未找到角点则退出
            if ret is not True:
                continue

            # 进一步优化角点坐标
            criteria = (cv2.TERM_CRITERIA_MAX_ITER +
                        cv2.TERM_CRITERIA_EPS, 30, 0.001)
            corners = cv2.cornerSubPix(
                gray_images[0], corners, (11, 11), (-1, -1), criteria)

            # 保存棋盘格角点图片
            if save_image:
                img = cv2.drawChessboardCorners(copy.deepcopy(
                    gray_images[0]), chessboard_size, corners, ret,)
                cv2.imwrite(save_path+'corners_{}'.format(i), img=img)

            # 生成角点在世界坐标系下的坐标点
            truth_locations = generate_truth_locations(
                chessboard_size, chessboard_cell_size)

            # 计算H矩阵
            H = recover_homographies(corners, truth_locations)
            H_s.append(H)



In [ ]:
Camera.static_calibrate()